In [1]:
from appJar import gui
import paho.mqtt.client as mqtt
import logging
import json
import stmpy
import sqlite3 as sq
import time

In [4]:
##Initialize Database, only run once
conn = sq.connect('Purr-fec.db')
c=conn.cursor()
c.execute('''CREATE TABLE cats
             (name TEXT PRIMARY KEY, rfid TEXT)''')
c.execute('''CREATE TABLE cordinates
             (cat TEXT, longditude REAL, latitude REAL, datetime TEXT)''')
conn.close()

In [2]:
MQTT_BROKER = 'mqtt.item.ntnu.no'
MQTT_PORT = 1883
MQTT_TOPIC_INPUT = 'ttm4115/team_1/server'
MQTT_COLLAR = 'ttm4115/team_1/collar'
MQTT_DOOR = 'ttm4115/team_1/door'

class CatStm:
    def conn_error(self):
        #self.server.user.receive_notification("Update from cat {} not received".format(name))
        print("Did not receive from ".format(self.name))
        
    def cat_in(self):
        print('cat in')
        command = {"command": "cat_in", "name": self.name}
        self.server.publish_command(command,MQTT_COLLAR)
    
    def cat_outside(self):
        command = {"command": "cat_outside", "name": self.name}
        self.server.publish_command(command,MQTT_COLLAR)

    def cat_outside_fence(self):
        command = {"command": "cat_outside_fence", "name": self.name}
        self.server.publish_command(command,MQTT_COLLAR)
    
    def start_tracking(self):
        command = {"command": "start_tracking", "name": self.name}
        self.server.publish_command(command,MQTT_COLLAR)
        
    def receive_data_home(self):
        data=server.latest_positions[self.name]
        if data[0] > server.lat_home+10 or data[0] < server.lat_home-10 or data[1] > server.lat_home+10 or data[1] <server.lat_home-10:
            return 's_outside'
        else:
            return 's_home'
        
        
    def receive_data_outside(self):
        data=server.latest_positions[self.name]
        if data[0] > server.lat_home+100 or data[0] < server.lat_home-100 or data[1] > server.lat_home+100 or data[1] <server.lat_home-100:
            #Notify
            return 's_outside_fence'
        else:
            return 's_outside'
        
    def receive_data_outside_fence(self):
        data=server.latest_positions[self.name]
        if data[0] > server.lat_home+100 or data[0] < server.lat_home-100 or data[1] > server.lat_home+100 or data[1] <server.lat_home-100:
            return 's_outside_fence'
        else:
            return 's_outside'
        
        
    def receive_data_tracking(self):
        print("NOTIFICATION")
    
    def __init__(self, name, server):
        self.name = name
        self.server = server
        
        t0 = {'source':'initial',
            'target':'cat_home',
            }
        #Home
        s_home = {'name':'cat_home',
            'entry':'start_timer("t",5000); cat_in'}
        
        t_btn_out = {'source':'cat_home',
            'target':'cat_outside',
            'trigger':'btn_outside'    
             }

        t_door_out = {'source':'cat_home',
            'target':'cat_outside',
            'trigger':'door_opened'
            }
        
        t_rec_data_home={'source':'cat_home',
            'trigger':'rec_data',
            'function':self.receive_data_home}
        
        t_self_home = {'source':'cat_home',
            'target':'cat_home',
            'trigger':'t',
            'effect':'conn_error'}
        
        
        #Outside
        s_outside = {'name':'cat_outside',
            'entry':'start_timer("t",5000); cat_outside'}
        
        t_btn_in = {'source':'cat_outside',
            'target':'cat_home',
            'trigger':'btn_inside'    
            }
        
        t_door_in  = {'source':'cat_outside',
            'target':'cat_home',
            'trigger':'door_opened'
            }
        
        t_self_outside = {'source':'cat_outside',
            'target':'cat_outside',
            'trigger':'t'
            }
        
        t_start_tracking = {'source': 'cat_outside',
            'target':'cat_tracking',
            'trigger':'start_tracking'}
        
        t_rec_data_outside={'source':'cat_outside',
            'trigger':'rec_data',
            'function':self.receive_data_outside}
        
        #Outside fence
        s_outside_fence = {'name':'cat_outside_fence',
            'entry':'start_timer("t",2000); cat_outside_fence'}
        
        t_self_outside_fence = {'source':'cat_outside_fence',
            'target':'cat_outside_fence',
            'effect':'conn_error'}
        
        t_rec_data_outside_fence={'source':'cat_outside_fence',
            'trigger':'rec_data',
            'function':self.receive_data_outside_fence}
        
        #Tracking
        s_tracking = {'name':'cat_tracking',
                'entry':'start_timer("t",2000);start_tracking'}
        
        t_self_tracking={'source':'cat_tracking',
            'target':'cat_tracking',
            'effect':'conn_error'}
        

        
        
        transitions=[t0, t_btn_out, t_btn_in, t_door_out, t_door_in, t_self_home,t_self_outside, t_start_tracking,]
        states=[s_home]
        self.stm = stmpy.Machine(name=name,transitions=transitions,obj=self,states=states) 

class Server:
    def register_cat(self,name,rfid):
        # SEND to door
        conn = sq.connect('Purr-fec.db')
        c=conn.cursor()
        c.execute('INSERT INTO cats VALUES(?,?)',(name,rfid))
        cat_stm = CatStm(name,self) 
        self.stm_driver.add_machine(cat_stm.stm)
        
        command={"rfid":rfid}
        self.publish_command(command,MQTT_DOOR)
        conn.commit()
        conn.close()
        
    def on_connect(self, client, userdata, flags, rc):
        print("Connected")
        
    def on_message(self, client, userdata, msg):
        try:
            payload = json.loads(msg.payload.decode("utf-8"))
        except Exception as err:
            print("invalid")
            return
        command = payload.get('command')
        if(command=='send_data'):
            name=payload.get('name')
            coordinates=(payload.get('lat'),payload.get('lon'))
            self.latest_positions[name]=coordinates
            print(self.latest_positions)
        if(command=='push_door'):
            rfid=payload.get('rfid')
            print(rfid)
            name = self.rfid2cat(rfid)
            print('swag')
            if name!=None: self.stm_driver.send('door_opened',name)
            

    
    def get_cat_names(self):
        conn = sq.connect('Purr-fec.db')
        c=conn.cursor()
        c.execute('SELECT name FROM cats')
        results = c.fetchall()
        cats = [ i[0] for i in results]
        conn.close()
        return cats

    def rfid2cat(self,rfid):
        conn = sq.connect('Purr-fec.db')
        c=conn.cursor()
        c.execute('SELECT name FROM cats WHERE rfid=' + rfid)
        name=c.fetchone()[0]
        conn.close()
        return name
    
    
    def publish_command(self,command,topic):
        payload = json.dumps(command)
        self.mqtt_client.publish(topic, payload=payload, qos=2)
    
    def __init__(self):
        self._logger = logging.getLogger(__name__)
        print('logging under name {}.'.format(__name__))
        self._logger.info('Starting Component')
        
        self.mqtt_client = mqtt.Client()
        self.mqtt_client.on_connect = self.on_connect
        self.mqtt_client.on_message = self.on_message
        self.mqtt_client.connect(MQTT_BROKER, MQTT_PORT)
        self.mqtt_client.subscribe(MQTT_TOPIC_INPUT)
        self.mqtt_client.loop_start()
        
        self.stm_driver = stmpy.Driver()
        self.stm_driver.start(keep_active=True)
        
        self.home_lon=0
        self.home_lat=0
        self.latest_positions={}
        #Starts sessions for already registered cats, if the cat is not home it will take some updates to converge
        cats = self.get_cat_names()
        for name in cats:
            cat_stm=CatStm(name,self)
            self.stm_driver.add_machine(cat_stm.stm)
        
        
    
    
    
    


# This class is supposed to emulate the browser connected via http. In this case the GUI calls the 
# relevant server-functions directly   
class User:
    def __init__(self,server):
        self.mqtt_client = mqtt.Client()
        self.server=server
        self.server.user = self

    def create_gui(self):
        self.app = gui()
        
        def req_pos():
            pass
        
        def new_cat_btn():
            self.app.showSubWindow('new_cat')
            
        def mon_cat_btn():
            cats = self.server.get_cat_names() #HTTP
            self.app.changeOptionBox('my_cats',cats)
            self.app.showSubWindow('monitor_cat')
            
        def send_new_cat():
            name = self.app.getEntry('Name')
            rfid = self.app.getEntry('RFID number')
            self.server.register_cat(name, rfid) #HTTP
            self.app.hideSubWindow('new_cat')
        
        def request_position():
            server.get_latest_positon()
            
        def settings_btn():
            self.app.showSubWindow('settings')
            
        def send_settings():
            home_lat = self.getEntry('home_lat')
            home_lon = self.getEntry('home_lon')
        
        """
        def receive_notification(self,message):
            self.app.SetMessage('notify_msg',message)
            self.app.showSubWindow('notification')
            print("self")
        """
            
        #Main
        self.app.addButton('New Cat',new_cat_btn)
        self.app.addButton('Follow a Cat', mon_cat_btn)
        self.app.addButton('Global Settings', settings_btn)

        #New cat subwindow
        self.app.startSubWindow('new_cat')
        self.app.addLabelEntry("Name")
        self.app.addLabelEntry("RFID number")
        self.app.addButton('Register',send_new_cat)
        self.app.stopSubWindow()
        
        #Monitor cat subwindow
        self.app.startSubWindow('monitor_cat')
        self.app.addLabel('Monitor Cat')
        self.app.addOptionBox('my_cats', [])
        self.app.addButton('Request Position',req_pos)        
        self.app.stopSubWindow()  
        
        #Global settings
        self.app.startSubWindow('settings')
        self.app.addEntry('home_lat')
        self.app.addEntry('home_lon')
        self.app.addButton('Update Settings',send_settings )
        self.app.stopSubWindow()
        
        """
        #Notification popup
        self.app.startSubWindow('notification')
        self.app.addMessage('notify_msg','')
        self.app.stopSubWindow()
        """
        self.app.go()
        

my_server = Server()
my_user = User(my_server)
my_user.create_gui()

2020-04-21 12:32:30,323 appJar:WARNING [799/_checkMode]: Interactive mode is not fully tested, some features might not work.
2020-04-21 12:32:30,335 appJar:WARNING [810/_checkMode]: iPython is not fully tested, some features might not work.


logging under name __main__.
Connected
cat in
Did not receive from 
cat in
Did not receive from 
cat in
12343
12343
Did not receive from 
cat in
12343
12343
12343
Did not receive from 
cat in
Did not receive from 
cat in
Did not receive from 
cat in
Did not receive from 
cat in
Did not receive from 
cat in
Did not receive from 
cat in
Did not receive from 
cat in
Did not receive from 
cat in
Did not receive from 
cat in
Did not receive from 
cat in
Did not receive from 
cat in
Did not receive from 
cat in
Did not receive from 
cat in
Did not receive from 
cat in
Did not receive from 
cat in
Did not receive from 
cat in
Did not receive from 
cat in
Did not receive from 
cat in
Did not receive from 
cat in
Did not receive from 
cat in
Did not receive from 
cat in
Did not receive from 
cat in
Did not receive from 
cat in
Did not receive from 
cat in
Did not receive from 
cat in
Did not receive from 
cat in
Did not receive from 
cat in
Did not receive from 
cat in
Did not receive from 
cat